In [4]:
import numpy as np
from scipy.stats import t as student_t


X = np.array([0.93, 1.44, 2.20, 3.28, 3.79, 2.17, 1.50, 2.18, 3.89, 0.60], float)
Y = np.array([1.61, 1.92, 1.89, 4.06, 3.51, 3.37, 1.32, 3.25, 3.59, 2.10], float)
n = len(X)

# Target mean and covariances (from the problem statement)
mu = np.array([2.0, 2.5])
Sigma_before = np.array([[1.0, 0.8],
                         [0.8, 1.0]], float)
Sigma_after  = np.array([[1.0, 0.2],
                         [0.2, 1.0]], float)


def spd_sqrt_and_invsqrt(S, eps=1e-12):
    S = 0.5*(S + S.T)                 # enforce symmetry
    w, Q = np.linalg.eigh(S)          # eigen-decomposition
    w = np.clip(w, eps, None)         # numerical safety
    S_sqrt = Q @ np.diag(np.sqrt(w)) @ Q.T
    S_invsqrt = Q @ np.diag(1/np.sqrt(w)) @ Q.T
    return S_sqrt, S_invsqrt

Sa_sqrt, _ = spd_sqrt_and_invsqrt(Sigma_after)
_, Sb_invsqrt = spd_sqrt_and_invsqrt(Sigma_before)

# Combined linear transform matrix:
# v' = mu + A (v - mu), where A = Sa^{1/2} Sb^{-1/2}
A = Sa_sqrt @ Sb_invsqrt

# Original vectors v_i as n×2 matrix
V = np.column_stack([X, Y])

# Deterministic transform of EXISTING data (not new independent data)
U = V - mu
U_new = (A @ U.T).T
V_new = U_new + mu

X_new = V_new[:, 0]
Y_new = V_new[:, 1]


D = X_new - Y_new
Dbar = D.mean()
sD = D.std(ddof=1)
t_paired = Dbar / (sD / np.sqrt(n))
p_paired = student_t.cdf(t_paired, df=n-1)   # left tail


Xbar = X_new.mean()
Ybar = Y_new.mean()
sX = X_new.std(ddof=1)
sY = Y_new.std(ddof=1)
SE_unmatched = np.sqrt(sX**2/n + sY**2/n)
t_unmatched = (Xbar - Ybar) / SE_unmatched
p_unmatched = student_t.cdf(t_unmatched, df=2*n-2)  # left tail

print("=== B.4 (cov 0.8 -> 0.2) results, NO resampling, only deterministic transform ===")
print(f"Paired:    t = {t_paired:.6f}, df = {n-1},   p(left) = {p_paired:.6f}")
print(f"Unmatched: t = {t_unmatched:.6f}, df = {2*n-2}, p(left) = {p_unmatched:.6f}")

=== B.4 (cov 0.8 -> 0.2) results, NO resampling, only deterministic transform ===
Paired:    t = -0.981321, df = 9,   p(left) = 0.176038
Unmatched: t = -0.889664, df = 18, p(left) = 0.192691
